Notebook created on 16 October 2021 by Chris Mulvey  
For NSS Data Science Bootcamp

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup as BS

### Working out a function to get all of the districts in a state

#### Reading in state and district info from a previous webscrape

In [3]:
districts = pd.read_csv('../data/district_info.csv')

districts.head()

,state,representatives
0,Alabama,7
1,Alaska,1
2,Arizona,9
3,Arkansas,4
4,California,53


### The URL has state and district number in it `id=TN02`

In [ ]:
url = 'https://www.opensecrets.org/races/summary?cycle=2020&id=TN02&spec=N'

In [ ]:
url = 'https://www.opensecrets.org/races/summary?cycle=2020&id='+ state + district'&spec=N'

### Function to get candidate info for a single state and district

In [13]:
def get_district_candidates(state, district):
    url = 'https://www.opensecrets.org/races/summary?cycle=2020&id='+ state + district + '&spec=N'
    response = requests.get(url)
    soup = BS(response.text)
    candidates = pd.read_html(str(soup.find('table')))[0]
    candidates['state'] = state
    candidates['district'] = district
    
    return candidates

In [16]:
get_district_candidates('TN', '03')

,Candidate,Raised,Spent,Cash on Hand,Last Report,state,district
0,Chuck Fleischmann (R) • Incumbent • Winner,"$1,051,653","$381,411","$1,880,341",12/31/2020,TN,03
1,Meg Gorman (D),"$85,843","$77,760","$8,083",12/31/2020,TN,03


In [ ]:
for index in districts.index:
    state = districts['state'][index]
    district = districts['representatives'][index]
    